In [1]:
from pprint import pprint
from collections import defaultdict
import json
import re
import sys
from typing import Union, List, Optional, Set, Tuple, Dict, Optional, Callable
from pprint import pprint
from IPython.display import display, HTML

import numpy as np
from lab.utils import shorten
import pandas as pd
from pathlib import Path
from machine_learning.analysis.dataframe import (
    pivot_rotate,

    slice_rows,
    slice_cols,
    sort_rows,
    sort_cols,
    aggregate,
    percentize,
    round,

    rename_index,
    rename_cols,
    rename_cells,

    isnan,
    to_latex,
    color_by_rank,
)
from machine_learning.analysis.series import (
    maybe_numeric_series,
)
from machine_learning.analysis.utils import (
    maybe_round,
)
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', 1000)

/home/acb11878tj/.pyenv/versions/3.8.5/envs/jupyter-analysis/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def find_result_paths(top_dir: Union[str, Path], regexps: Optional[List[str]] = None) -> List[str]:
    regexps = regexps or []
    top_dir = Path(top_dir)
    return [
        str(path) for path in top_dir.glob('**/results.tsv')
        if all(re.match(regexp, str(path)) for regexp in regexps)
    ]

In [3]:
def name_method(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
            
    df[COL_METHOD] = df.apply(
        lambda row: '__'.join(str(row[col]) for col in METHOD_DEFINE_COLS),
        axis=1,
    )

    df = sort_cols(df, [COL_METHOD, '.*'])

    return df

def name_task(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    df[COL_TASK] = df.apply(
        lambda row: '__'.join(str(row[col]) for col in TASK_DEFINE_COLS),
        axis=1,
    )

    df = sort_cols(df, [COL_TASK, '.*'])
    
    return df

def prettify_df(df: pd.DataFrame) -> pd.DataFrame:
    df = percentize(df)
    df = round(df)
    return df

In [4]:
COL_DATASET = 'dataset_uname'
COL_LEARNING = 'learning'
COL_MODEL_NAME_OR_PATH = 'model_name_or_path'
COL_LRATE = 'learning_rate'

TASK_DEFINE_COLS = [COL_DATASET, COL_LEARNING]
COL_TASK = 'task'

METHOD_DEFINE_COLS = [COL_MODEL_NAME_OR_PATH]
COL_METHOD = 'method'
METHOD_RENAMES = {
    '^retrieva-jp/t5-base-long$': 'retrieva-t5-base',

    '^line-corporation/japanese-large-lm-1.7b$': 'line-1B',
    '^line-corporation/japanese-large-lm-1.7b-instruction-sft$': 'line-1B-instruct',
    
    '^cyberagent/open-calm-1b$': 'calm-1B',
    '^cyberagent/open-calm-7b$': 'calm-7B',
    
    '^stabilityai/japanese-stablelm-base-alpha-7b$': 'stablelm-7B',
    
    '^elyza/ELYZA-japanese-Llama-2-7b-fast-instruct$': 'elyza-7B-instruct',
    
    '^matsuo-lab/weblab-10b$$': 'weblab-10B',
    '^matsuo-lab/weblab-10b-instruction-sft$': 'weblab-10B-instruct',
}

COL_LRATE = 'learning_rate'
LRATE = 0.0001

METRIC_RENAMES = {
    'eval/extr_stps.D-all.proof_accuracy.zero_one': 'prf_acc.extr',
    # 'eval/strct.D-all.proof_accuracy.zero_one': 'prf_acc.strct',
    # 'eval/strct.D-all.answer_accuracy': 'ans.acc',
}
METRIC_NAMES = list(METRIC_RENAMES.values())

TASK_RENAMES = {
    # '^20230826.jpn.D3__nan$': 'D3.full',
    # '^20230826.jpn.D8__nan$': 'D8.full',
    
    '^20230904.jpn.D1.wo_brnch.wo_dstrct__LLM_FS.shot-10$': 'D1-.10',
    '^20230904.jpn.D1.wo_brnch.wo_dstrct__LLM_FS.shot-100$': 'D1-.100',
    '^20230904.jpn.D1.wo_brnch.wo_dstrct__LLM_FS.shot-1000$': 'D1-.1000',
    
    # '^20230904.jpn.D1.wo_brnch__LLM_FS.shot-10$': 'D1-.10',
    # '^20230904.jpn.D1.wo_brnch__LLM_FS.shot-100$': 'D1-.100',
    # '^20230904.jpn.D1.wo_brnch__LLM_FS.shot-1000$': 'D1-.1000',
    
    '^20230904.jpn.D1__LLM_FS.shot-10$': 'D1.10',
    '^20230904.jpn.D1__LLM_FS.shot-100$': 'D1.100',
    '^20230904.jpn.D1__LLM_FS.shot-1000$': 'D1.1000',
    
    '^20230904.jpn.D3__LLM_FS.shot-10$': 'D3.10',
    '^20230904.jpn.D3__LLM_FS.shot-100$': 'D3.100',
    '^20230904.jpn.D3__LLM_FS.shot-1000$': 'D3.1000',
}
TASK_NAMES = list(TASK_RENAMES.values())
MAJOR_TASK = TASK_NAMES[1]

COLOR_SCALE_LOWER = 0
COLOR_SCALE_UPPER = 100
COLOR_PARETTE_LOWER = 3
COLOR_PARETTE_UPPER = 70

In [5]:
# _TOP_DIR = '../outputs/02.aggregate_tf_results.py/20230826.jpn/'
# _TOP_DIR = '../outputs/02.aggregate_tf_results.py/20230901.overfit/'
# _TOP_DIR = '../outputs/02.aggregate_tf_results.py/20230904.LLM_FS/'
# _TOP_DIR = '../outputs/02.aggregate_tf_results.py/20230905.LLM_FS/'

_TOP_DIR = '../outputs/02.aggregate_tf_results.py/20230910.preliminary'

In [6]:
result_paths = find_result_paths(_TOP_DIR)
if len(result_paths) == 0:
    raise Exception(f'Results not found under {_TOP_DIR}')
elif len(result_paths) == 1:
    results_path = result_paths[0]
else:
    print('Choose the result fomr the following paths:')
    pprint(result_paths)
    results_path = input('path = ')
    


In [7]:
master_df = pd.read_csv(results_path, sep='\t')
master_df

,dataset_uname,learning,model_name_or_path,seed,learning_rate,base_config_name,generation_max_proof_steps,generation_num_beams,generation_top_k,gradient_accumulation_steps,lm_type,lora,max_grad_norm,max_predict_samples,max_proof_steps,max_source_length,max_steps,max_target_length,max_train_samples,per_device_eval_batch_size,per_device_train_batch_size,proof_sampling,sample_negative_proof,source_prefix,tokenizer_padding,warmup_steps,eval/extr_stps.D-0.proof_accuracy.zero_one,eval/extr_stps.D-1.proof_accuracy.zero_one,eval/extr_stps.D-2.proof_accuracy.zero_one,eval/extr_stps.D-3.proof_accuracy.zero_one,eval/extr_stps.D-4.proof_accuracy.zero_one,eval/extr_stps.D-5.proof_accuracy.zero_one,eval/extr_stps.D-6.proof_accuracy.zero_one,eval/extr_stps.D-7.proof_accuracy.zero_one,eval/extr_stps.D-8.proof_accuracy.zero_one,eval/extr_stps.D-None.proof_accuracy.zero_one,eval/extr_stps.D-all.proof_accuracy.zero_one,eval/strct.D-0.proof_accuracy.zero_one,eval/strct.D-1.proof_accuracy.zero_one,eval/strct.D-2.proof_accuracy.zero_one,eval/strct.D-3.proof_accuracy.zero_one,eval/strct.D-4.proof_accuracy.zero_one,eval/strct.D-5.proof_accuracy.zero_one,eval/strct.D-6.proof_accuracy.zero_one,eval/strct.D-7.proof_accuracy.zero_one,eval/strct.D-8.proof_accuracy.zero_one,eval/strct.D-None.proof_accuracy.zero_one,eval/strct.D-all.proof_accuracy.zero_one,eval/extr_stps.D-0.answer_accuracy,eval/extr_stps.D-1.answer_accuracy,eval/extr_stps.D-2.answer_accuracy,eval/extr_stps.D-3.answer_accuracy,eval/extr_stps.D-4.answer_accuracy,eval/extr_stps.D-5.answer_accuracy,eval/extr_stps.D-6.answer_accuracy,eval/extr_stps.D-7.answer_accuracy,eval/extr_stps.D-8.answer_accuracy,eval/extr_stps.D-None.answer_accuracy,eval/extr_stps.D-all.answer_accuracy,eval/strct.D-0.answer_accuracy,eval/strct.D-1.answer_accuracy,eval/strct.D-2.answer_accuracy,eval/strct.D-3.answer_accuracy,eval/strct.D-4.answer_accuracy,eval/strct.D-5.answer_accuracy,eval/strct.D-6.answer_accuracy,eval/strct.D-7.answer_accuracy,eval/strct.D-8.answer_accuracy,eval/strct.D-None.answer_accuracy,eval/strct.D-all.answer_accuracy
0,20230904.jpn.D1.wo_brnch,LLM_FS.shot-100,matsuo-lab/weblab-10b,0,0.0001,FLNLcorpus.20220827.base,20,1,10,4,causal,True,0.5,0,30,2000,170,2000,100,1,2,all_at_once,False,Solve FLD task:,longest,17,0.000000,0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.100000,0.066667,0.000000,0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.100000,0.066667,0.333333,0.529412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.100000,0.366667,0.333333,0.529412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.100000,0.366667
1,20230904.jpn.D1.wo_brnch,LLM_FS.shot-100,matsuo-lab/weblab-10b-instruction-sft,0,0.0001,FLNLcorpus.20220827.base,20,1,10,4,causal,True,0.5,0,30,2000,170,2000,100,1,2,all_at_once,False,Solve FLD task:,longest,17,0.333333,0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.400000,0.200000,0.333333,0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.400000,0.200000,0.666667,0.705882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.400000,0.600000,0.666667,0.705882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.400000,0.600000
2,20230904.jpn.D1.wo_brnch,LLM_FS.shot-100,cyberagent/open-calm-7b,0,0.0001,FLNLcorpus.20220827.base,20,1,10,4,causal,True,0.5,0,30,2000,170,2000,100,1,2,all_at_once,False,Solve FLD task:,longest,17,0.000000,0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.300000,0.133333,0.000000,0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.300000,0.133333,0.666667,0.529412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.300000,0.466667,0.666667,0.529412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.300000,0.466667
3,20230904.jpn.D1.wo_brnch,LLM_FS.shot-100,line-corporation/japanese-large-lm-1.7b-instruction-sft,0,0.0001,FLNLcorpus.20220827.base,20,1,10,1,causal,True,0.5,0,30,2000,170,2000,100,2,8,all_at_once,False,Solve FLD task:,longest,17,0.000000,0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.100000,0.066667,0.000000,0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.100000,0.066667,1.000000,0.647059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.100000,0.500000,1.000000,0.647059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.100000,0.500000
4,20230904.jpn.D1.wo_brnch,LLM_FS.shot-100,line-corp

In [8]:
df = name_method(master_df)
df = rename_cells(df, [COL_METHOD], METHOD_RENAMES)
df = sort_rows(df, COL_METHOD, [f'^{name}$' for name in METHOD_RENAMES.values()])

df = name_task(df)
df = rename_cells(df, [COL_TASK], TASK_RENAMES)


df = rename_cols(df, METRIC_RENAMES)
                 
df = slice_rows(df, lambda row: row[COL_LRATE] == LRATE)
df = aggregate(
    df,
    [COL_METHOD, COL_TASK],
    {metric_name: lambda vals: np.mean(vals) for metric_name in METRIC_NAMES},
)

df

,method,task,dataset_uname,learning,model_name_or_path,seed,learning_rate,base_config_name,generation_max_proof_steps,generation_num_beams,generation_top_k,gradient_accumulation_steps,lm_type,lora,max_grad_norm,max_predict_samples,max_proof_steps,max_source_length,max_steps,max_target_length,max_train_samples,per_device_eval_batch_size,per_device_train_batch_size,proof_sampling,sample_negative_proof,source_prefix,tokenizer_padding,warmup_steps,eval/extr_stps.D-0.proof_accuracy.zero_one,eval/extr_stps.D-1.proof_accuracy.zero_one,eval/extr_stps.D-2.proof_accuracy.zero_one,eval/extr_stps.D-3.proof_accuracy.zero_one,eval/extr_stps.D-4.proof_accuracy.zero_one,eval/extr_stps.D-5.proof_accuracy.zero_one,eval/extr_stps.D-6.proof_accuracy.zero_one,eval/extr_stps.D-7.proof_accuracy.zero_one,eval/extr_stps.D-8.proof_accuracy.zero_one,eval/extr_stps.D-None.proof_accuracy.zero_one,prf_acc.extr,eval/strct.D-0.proof_accuracy.zero_one,eval/strct.D-1.proof_accuracy.zero_one,eval/strct.D-2.proof_accuracy.zero_one,eval/strct.D-3.proof_accuracy.zero_one,eval/strct.D-4.proof_accuracy.zero_one,eval/strct.D-5.proof_accuracy.zero_one,eval/strct.D-6.proof_accuracy.zero_one,eval/strct.D-7.proof_accuracy.zero_one,eval/strct.D-8.proof_accuracy.zero_one,eval/strct.D-None.proof_accuracy.zero_one,eval/strct.D-all.proof_accuracy.zero_one,eval/extr_stps.D-0.answer_accuracy,eval/extr_stps.D-1.answer_accuracy,eval/extr_stps.D-2.answer_accuracy,eval/extr_stps.D-3.answer_accuracy,eval/extr_stps.D-4.answer_accuracy,eval/extr_stps.D-5.answer_accuracy,eval/extr_stps.D-6.answer_accuracy,eval/extr_stps.D-7.answer_accuracy,eval/extr_stps.D-8.answer_accuracy,eval/extr_stps.D-None.answer_accuracy,eval/extr_stps.D-all.answer_accuracy,eval/strct.D-0.answer_accuracy,eval/strct.D-1.answer_accuracy,eval/strct.D-2.answer_accuracy,eval/strct.D-3.answer_accuracy,eval/strct.D-4.answer_accuracy,eval/strct.D-5.answer_accuracy,eval/strct.D-6.answer_accuracy,eval/strct.D-7.answer_accuracy,eval/strct.D-8.answer_accuracy,eval/strct.D-None.answer_accuracy,eval/strct.D-all.answer_accuracy
"(line-1B, 20230904.jpn.D1.wo_brnch__LLM_FS.shot-100)",line-1B,20230904.jpn.D1.wo_brnch__LLM_FS.shot-100,20230904.jpn.D1.wo_brnch,LLM_FS.shot-100,line-corporation/japanese-large-lm-1.7b,0,0.0001,FLNLcorpus.20220827.base,20,1,10,1,causal,True,0.5,0,30,2000,170,2000,100,2,8,all_at_once,False,Solve FLD task:,longest,17,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1.000000,0.411765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.333333,1.000000,0.411765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.333333
"(line-1B, 20230904.jpn.D1.wo_brnch__LLM_FS.shot-10)",line-1B,20230904.jpn.D1.wo_brnch__LLM_FS.shot-10,20230904.jpn.D1.wo_brnch,LLM_FS.shot-10,line-corporation/japanese-large-lm-1.7b,0,0.0001,FLNLcorpus.20220827.base,20,1,10,1,causal,True,0.5,0,30,2000,70,2000,10,2,8,all_at_once,False,Solve FLD task:,longest,7,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
"(line-1B, 20230904.jpn.D1.wo_brnch__LLM_FS.shot-1000)",line-1B,20230904.jpn.D1.wo_brnch__LLM_FS.shot-1000,20230904.jpn.D1.wo_brnch,LLM_FS.shot-1000,line-corporation/japanese-large-lm-1.7b,0,0.0001,FLNLcorpus.20220827.base,20,1,10,1,causal,True,0.5,0,30,2000,320,2000,1000,2,8,all_at_once,False,Solve FLD task:,longest,32,0.000000,0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.033333,0.000000,0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.033333,0.333333,0.647059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.400000,0.333333,0.647059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.400000
"(line-1B, D1-.100)",line-1B,D1-.100,20230904.jpn.D1.wo_brnch.wo_dstrct,LLM_FS.shot-100,line-corporation/japanese-large-lm-1.7b,0,0.0001,FLNLcorpus.20220827.base,20,1,10,1,causal,True,0.5,0,30,2000,170,2000,100,2,8,a

In [9]:
for task in df[COL_TASK].unique():
    print("'" + task + "',")

'20230904.jpn.D1.wo_brnch__LLM_FS.shot-100',
'20230904.jpn.D1.wo_brnch__LLM_FS.shot-10',
'20230904.jpn.D1.wo_brnch__LLM_FS.shot-1000',
'D1-.100',
'D1-.10',
'D1-.1000',
'D1.100',
'D1.10',
'D1.1000',
'D3.100',
'D3.10',
'D3.1000',


In [10]:
task_dfs: Dict[str, pd.DataFrame] = {}
for task_name in TASK_NAMES:
    task_dfs[task_name] = slice_rows(
        df,
        lambda row: row[COL_TASK] == task_name
    )

task_dfs[MAJOR_TASK]

,method,task,dataset_uname,learning,model_name_or_path,seed,learning_rate,base_config_name,generation_max_proof_steps,generation_num_beams,generation_top_k,gradient_accumulation_steps,lm_type,lora,max_grad_norm,max_predict_samples,max_proof_steps,max_source_length,max_steps,max_target_length,max_train_samples,per_device_eval_batch_size,per_device_train_batch_size,proof_sampling,sample_negative_proof,source_prefix,tokenizer_padding,warmup_steps,eval/extr_stps.D-0.proof_accuracy.zero_one,eval/extr_stps.D-1.proof_accuracy.zero_one,eval/extr_stps.D-2.proof_accuracy.zero_one,eval/extr_stps.D-3.proof_accuracy.zero_one,eval/extr_stps.D-4.proof_accuracy.zero_one,eval/extr_stps.D-5.proof_accuracy.zero_one,eval/extr_stps.D-6.proof_accuracy.zero_one,eval/extr_stps.D-7.proof_accuracy.zero_one,eval/extr_stps.D-8.proof_accuracy.zero_one,eval/extr_stps.D-None.proof_accuracy.zero_one,prf_acc.extr,eval/strct.D-0.proof_accuracy.zero_one,eval/strct.D-1.proof_accuracy.zero_one,eval/strct.D-2.proof_accuracy.zero_one,eval/strct.D-3.proof_accuracy.zero_one,eval/strct.D-4.proof_accuracy.zero_one,eval/strct.D-5.proof_accuracy.zero_one,eval/strct.D-6.proof_accuracy.zero_one,eval/strct.D-7.proof_accuracy.zero_one,eval/strct.D-8.proof_accuracy.zero_one,eval/strct.D-None.proof_accuracy.zero_one,eval/strct.D-all.proof_accuracy.zero_one,eval/extr_stps.D-0.answer_accuracy,eval/extr_stps.D-1.answer_accuracy,eval/extr_stps.D-2.answer_accuracy,eval/extr_stps.D-3.answer_accuracy,eval/extr_stps.D-4.answer_accuracy,eval/extr_stps.D-5.answer_accuracy,eval/extr_stps.D-6.answer_accuracy,eval/extr_stps.D-7.answer_accuracy,eval/extr_stps.D-8.answer_accuracy,eval/extr_stps.D-None.answer_accuracy,eval/extr_stps.D-all.answer_accuracy,eval/strct.D-0.answer_accuracy,eval/strct.D-1.answer_accuracy,eval/strct.D-2.answer_accuracy,eval/strct.D-3.answer_accuracy,eval/strct.D-4.answer_accuracy,eval/strct.D-5.answer_accuracy,eval/strct.D-6.answer_accuracy,eval/strct.D-7.answer_accuracy,eval/strct.D-8.answer_accuracy,eval/strct.D-None.answer_accuracy,eval/strct.D-all.answer_accuracy
"(line-1B, D1-.100)",line-1B,D1-.100,20230904.jpn.D1.wo_brnch.wo_dstrct,LLM_FS.shot-100,line-corporation/japanese-large-lm-1.7b,0,0.0001,FLNLcorpus.20220827.base,20,1,10,1,causal,True,0.5,0,30,2000,170,2000,100,2,8,all_at_once,False,Solve FLD task:,longest,17,0.0,0.476190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.571429,0.466667,0.0,0.476190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.571429,0.466667,0.0,0.476190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.571429,0.466667,0.0,0.476190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.571429,0.466667
"(line-1B-instruct, D1-.100)",line-1B-instruct,D1-.100,20230904.jpn.D1.wo_brnch.wo_dstrct,LLM_FS.shot-100,line-corporation/japanese-large-lm-1.7b-instruction-sft,0,0.0001,FLNLcorpus.20220827.base,20,1,10,1,causal,True,0.5,0,30,2000,170,2000,100,2,8,all_at_once,False,Solve FLD task:,longest,17,0.0,0.523810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.714286,0.533333,0.0,0.523810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.714286,0.533333,0.0,0.619048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.714286,0.600000,0.0,0.619048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.714286,0.600000
"(calm-1B, D1-.100)",calm-1B,D1-.100,20230904.jpn.D1.wo_brnch.wo_dstrct,LLM_FS.shot-100,cyberagent/open-calm-1b,0,0.0001,FLNLcorpus.20220827.base,20,1,10,1,causal,True,0.5,0,30,2000,170,2000,100,2,8,all_at_once,False,Solve FLD task:,longest,17,0.0,0.714286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.428571,0.600000,0.0,0.714286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.428571,0.600000,0.0,0.761905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.428571,0.633333,0.0,0.761905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.428571,0.633333
"(calm-7B, D1-.100)",calm-7B,D1-.100,20230904.jpn.D1.wo_brnch.wo_dstrct,LLM_FS.shot-100,cyberagent/open-calm-7b,0,0.0001,FLNLcorpus.20220827.base,20,1,10,4,causal,True,0.5,0,30,2000,170,2000,100,1,2,all_at_once,False,Solve FLD task:,longest,17,0.0,0.428571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.571429,0.433333,0.0,0.428571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.571429,0.433333,0.0,0.428571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.571429,0.433333,0.0,0.428

In [11]:

metric_dfs: Dict[str, pd.DataFrame] = {}
for task_name, task_df in task_dfs.items():
    metric_dfs[task_name] = slice_cols(task_df, [COL_TASK, COL_METHOD] + METRIC_NAMES)

for task_name in TASK_NAMES:
    print('\n\n')
    print(f'========================= {task_name} ========================')
    display(metric_dfs[task_name])




========================= D1-.10 ========================


,task,method,prf_acc.extr
"(line-1B, D1-.10)",D1-.10,line-1B,0.000000
"(line-1B-instruct, D1-.10)",D1-.10,line-1B-instruct,0.266667
"(calm-1B, D1-.10)",D1-.10,calm-1B,0.233333
"(calm-7B, D1-.10)",D1-.10,calm-7B,0.133333
"(elyza-7B-instruct, D1-.10)",D1-.10,elyza-7B-instruct,0.300000
"(weblab-10B, D1-.10)",D1-.10,weblab-10B,0.200000
"(weblab-10B-instruct, D1-.10)",D1-.10,weblab-10B-instruct,0.166667





========================= D1-.100 ========================


,task,method,prf_acc.extr
"(line-1B, D1-.100)",D1-.100,line-1B,0.466667
"(line-1B-instruct, D1-.100)",D1-.100,line-1B-instruct,0.533333
"(calm-1B, D1-.100)",D1-.100,calm-1B,0.600000
"(calm-7B, D1-.100)",D1-.100,calm-7B,0.433333
"(elyza-7B-instruct, D1-.100)",D1-.100,elyza-7B-instruct,0.733333
"(weblab-10B, D1-.100)",D1-.100,weblab-10B,0.466667
"(weblab-10B-instruct, D1-.100)",D1-.100,weblab-10B-instruct,0.800000





========================= D1-.1000 ========================


...

,task,method,prf_acc.extr
"(line-1B, D3.10)",D3.10,line-1B,0.000000
"(line-1B-instruct, D3.10)",D3.10,line-1B-instruct,0.033333
"(calm-1B, D3.10)",D3.10,calm-1B,0.166667
"(calm-7B, D3.10)",D3.10,calm-7B,0.000000
"(elyza-7B-instruct, D3.10)",D3.10,elyza-7B-instruct,0.200000
"(weblab-10B, D3.10)",D3.10,weblab-10B,0.000000
"(weblab-10B-instruct, D3.10)",D3.10,weblab-10B-instruct,0.000000





========================= D3.100 ========================


,task,method,prf_acc.extr
"(line-1B, D3.100)",D3.100,line-1B,0.066667
"(line-1B-instruct, D3.100)",D3.100,line-1B-instruct,0.310345
"(calm-1B, D3.100)",D3.100,calm-1B,0.133333
"(calm-7B, D3.100)",D3.100,calm-7B,0.166667
"(elyza-7B-instruct, D3.100)",D3.100,elyza-7B-instruct,0.333333
"(weblab-10B, D3.100)",D3.100,weblab-10B,0.000000
"(weblab-10B-instruct, D3.100)",D3.100,weblab-10B-instruct,0.100000





========================= D3.1000 ========================


,task,method,prf_acc.extr
"(line-1B, D3.1000)",D3.1000,line-1B,0.000000
"(line-1B-instruct, D3.1000)",D3.1000,line-1B-instruct,0.000000
"(calm-1B, D3.1000)",D3.1000,calm-1B,0.000000
"(calm-7B, D3.1000)",D3.1000,calm-7B,0.033333
"(elyza-7B-instruct, D3.1000)",D3.1000,elyza-7B-instruct,0.466667
"(weblab-10B, D3.1000)",D3.1000,weblab-10B,0.100000
"(weblab-10B-instruct, D3.1000)",D3.1000,weblab-10B-instruct,0.166667


In [12]:

pretty_dfs: Dict[str, pd.DataFrame] = {}

for task_name, metric_df in metric_dfs.items():
    pretty_df = prettify_df(metric_df)
    pretty_df = rename_cols(pretty_df, METRIC_RENAMES)
    
    pretty_df.index = pretty_df[COL_METHOD]
    pretty_df = pretty_df.drop(columns=[COL_METHOD])
    
    pretty_df = color_by_rank(pretty_df, 'col',
                              scale_lower=COLOR_SCALE_LOWER,
                              scale_upper=COLOR_SCALE_UPPER,
                              color_lower=COLOR_PARETTE_LOWER,
                              color_upper=COLOR_PARETTE_UPPER)
    
    pretty_dfs[task_name] = pretty_df


for task_name in TASK_NAMES:
    print('\n\n')
    print(f'========================= {task_name} ========================')
    display(pretty_dfs[task_name])




========================= D1-.10 ========================


,task,prf_acc.extr
method,,
line-1B,D1-.10,\cellcolor{blue!3} 0.0
line-1B-instruct,D1-.10,\cellcolor{blue!20} 26.7
calm-1B,D1-.10,\cellcolor{blue!18} 23.3
calm-7B,D1-.10,\cellcolor{blue!11} 13.3
elyza-7B-instruct,D1-.10,\cellcolor{blue!23} 30.0
weblab-10B,D1-.10,\cellcolor{blue!16} 20.0
weblab-10B-instruct,D1-.10,\cellcolor{blue!14} 16.7





========================= D1-.100 ========================


,task,prf_acc.extr
method,,
line-1B,D1-.100,\cellcolor{blue!34} 46.7
line-1B-instruct,D1-.100,\cellcolor{blue!38} 53.3
calm-1B,D1-.100,\cellcolor{blue!43} 60.0
calm-7B,D1-.100,\cellcolor{blue!32} 43.3
elyza-7B-instruct,D1-.100,\cellcolor{blue!52} 73.3
weblab-10B,D1-.100,\cellcolor{blue!34} 46.7
weblab-10B-instruct,D1-.100,\cellcolor{blue!56} 80.0





========================= D1-.1000 ========================


...

,task,prf_acc.extr
method,,
line-1B,D3.10,\cellcolor{blue!3} 0.0
line-1B-instruct,D3.10,\cellcolor{blue!5} 3.3
calm-1B,D3.10,\cellcolor{blue!14} 16.7
calm-7B,D3.10,\cellcolor{blue!3} 0.0
elyza-7B-instruct,D3.10,\cellcolor{blue!16} 20.0
weblab-10B,D3.10,\cellcolor{blue!3} 0.0
weblab-10B-instruct,D3.10,\cellcolor{blue!3} 0.0





========================= D3.100 ========================


,task,prf_acc.extr
method,,
line-1B,D3.100,\cellcolor{blue!7} 6.7
line-1B-instruct,D3.100,\cellcolor{blue!23} 31.0
calm-1B,D3.100,\cellcolor{blue!11} 13.3
calm-7B,D3.100,\cellcolor{blue!14} 16.7
elyza-7B-instruct,D3.100,\cellcolor{blue!25} 33.3
weblab-10B,D3.100,\cellcolor{blue!3} 0.0
weblab-10B-instruct,D3.100,\cellcolor{blue!9} 10.0





========================= D3.1000 ========================


,task,prf_acc.extr
method,,
line-1B,D3.1000,\cellcolor{blue!3} 0.0
line-1B-instruct,D3.1000,\cellcolor{blue!3} 0.0
calm-1B,D3.1000,\cellcolor{blue!3} 0.0
calm-7B,D3.1000,\cellcolor{blue!5} 3.3
elyza-7B-instruct,D3.1000,\cellcolor{blue!34} 46.7
weblab-10B,D3.1000,\cellcolor{blue!9} 10.0
weblab-10B-instruct,D3.1000,\cellcolor{blue!14} 16.7


In [13]:
def horizontal_concat(dfs: List[pd.DataFrame]) -> pd.DataFrame:
    dfs = [df.copy() for df in dfs]
    # align index for horizontal concat
    # for df in dfs:
    #     df.index = range(len(df))
    return pd.concat(dfs, axis=1)

colored_concat_df = horizontal_concat(
    [pretty_df for task_name, pretty_df in pretty_dfs.items()]
)
colored_concat_df = colored_concat_df.drop(columns=[COL_TASK], axis=1)

print('    '.join([task_name for task_name in pretty_dfs.keys()]))
colored_concat_df

D1-.10    D1-.100    D1-.1000    D1.10    D1.100    D1.1000    D3.10    D3.100    D3.1000


,prf_acc.extr,prf_acc.extr,prf_acc.extr,prf_acc.extr,prf_acc.extr,prf_acc.extr,prf_acc.extr,prf_acc.extr,prf_acc.extr
method,,,,,,,,,
line-1B,\cellcolor{blue!3} 0.0,\cellcolor{blue!34} 46.7,\cellcolor{blue!45} 63.3,\cellcolor{blue!3} 0.0,\cellcolor{blue!9} 10.0,\cellcolor{blue!16} 20.0,\cellcolor{blue!3} 0.0,\cellcolor{blue!7} 6.7,\cellcolor{blue!3} 0.0
line-1B-instruct,\cellcolor{blue!20} 26.7,\cellcolor{blue!38} 53.3,\cellcolor{blue!45} 63.3,\cellcolor{blue!5} 3.3,\cellcolor{blue!7} 6.7,\cellcolor{blue!7} 6.7,\cellcolor{blue!5} 3.3,\cellcolor{blue!23} 31.0,\cellcolor{blue!3} 0.0
calm-1B,\cellcolor{blue!18} 23.3,\cellcolor{blue!43} 60.0,\cellcolor{blue!49} 70.0,\cellcolor{blue!5} 3.3,\cellcolor{blue!5} 3.3,\cellcolor{blue!5} 3.3,\cellcolor{blue!14} 16.7,\cellcolor{blue!11} 13.3,\cellcolor{blue!3} 0.0
calm-7B,\cellcolor{blue!11} 13.3,\cellcolor{blue!32} 43.3,\cellcolor{blue!43} 60.0,\cellcolor{blue!3} 0.0,\cellcolor{blue!9} 10.0,\cellcolor{blue!3} 0.0,\cellcolor{blue!3} 0.0,\cellcolor{blue!14} 16.7,\cellcolor{blue!5} 3.3
elyza-7B-instruct,\cellcolor{blue!23} 30.0,\cellcolor{blue!52} 73.3,\cellcolor{blue!43} 60.0,\cellcolor{blue!5} 3.3,\cellcolor{blue!14} 16.7,\cellcolor{blue!32} 43.3,\cellcolor{blue!16} 20.0,\cellcolor{blue!25} 33.3,\cellcolor{blue!34} 46.7
weblab-10B,\cellcolor{blue!16} 20.0,\cellcolor{blue!34} 46.7,\cellcolor{blue!67} 96.7,\cellcolor{blue!5} 3.3,\cellcolor{blue!5} 3.3,\cellcolor{blue!16} 20.0,\cellcolor{blue!3} 0.0,\cellcolor{blue!3} 0.0,\cellcolor{blue!9} 10.0
weblab-10B-instruct,\cellcolor{blue!14} 16.7,\cellcolor{blue!56} 80.0,\cellcolor{blue!67} 96.7,\cellcolor{blue!5} 3.3,\cellcolor{blue!9} 10.0,\cellcolor{blue!29} 40.0,\cellcolor{blue!3} 0.0,\cellcolor{blue!9} 10.0,\cellcolor{blue!14} 16.7


In [14]:
def add_gpt_row(latex_str: str) -> str:
    lines: List[str] = []
    for line in latex_str.split('\n'):
        if re.match(f'^ *{COL_METHOD}.*', line):
            gpt_line = re.sub('\&[^\&]*', '& - ', line).replace(COL_METHOD, 'GPT-4') + '  \\\\'
            lines.append(line)
            lines.append(gpt_line)
        else:
            lines.append(line)
    return '\n'.join(lines)
    
latex_str = to_latex(colored_concat_df, with_index=True)
print(add_gpt_row(latex_str))

\begin{tabular}{llllllllll}
\toprule
 & prf\_acc.extr & prf\_acc.extr & prf\_acc.extr & prf\_acc.extr & prf\_acc.extr & prf\_acc.extr & prf\_acc.extr & prf\_acc.extr & prf\_acc.extr \\
method &  &  &  &  &  &  &  &  &  \\
GPT-4 & - & - & - & - & - & - & - & - & -   \\
\midrule
line-1B & \cellcolor{blue!3} 0.0 & \cellcolor{blue!34} 46.7 & \cellcolor{blue!45} 63.3 & \cellcolor{blue!3} 0.0 & \cellcolor{blue!9} 10.0 & \cellcolor{blue!16} 20.0 & \cellcolor{blue!3} 0.0 & \cellcolor{blue!7} 6.7 & \cellcolor{blue!3} 0.0 \\
line-1B-instruct & \cellcolor{blue!20} 26.7 & \cellcolor{blue!38} 53.3 & \cellcolor{blue!45} 63.3 & \cellcolor{blue!5} 3.3 & \cellcolor{blue!7} 6.7 & \cellcolor{blue!7} 6.7 & \cellcolor{blue!5} 3.3 & \cellcolor{blue!23} 31.0 & \cellcolor{blue!3} 0.0 \\
calm-1B & \cellcolor{blue!18} 23.3 & \cellcolor{blue!43} 60.0 & \cellcolor{blue!49} 70.0 & \cellcolor{blue!5} 3.3 & \cellcolor{blue!5} 3.3 & \cellcolor{blue!5} 3.3 & \cellcolor{blue!14} 16.7 & \cellcolor{blue!11} 13.3 & \cellc

In [15]:
def task_name_to_shot(task_name: str) -> str:
    return task_name.split('.')[-1]

def task_name_to_dataset_name(task_name: str) -> str:
    return '.'.join(task_name.split('.')[:-1])

num_metrics = len(METRIC_RENAMES)
num_task = len(pretty_dfs)

dataset_names = []
for task_name in pretty_dfs: 
    dataset_name = task_name_to_dataset_name(task_name)
    if dataset_name not in dataset_names:
        dataset_names.append(dataset_name)
num_datasets = len(dataset_names)
num_shot_settings = int(num_task / num_datasets)

dataset_row = '{}    &    ' + '  &  '.join([f'\multicolumn{{{num_shot_settings * num_metrics}}}{{c}}{{{dataset_name}}}'
                                            for dataset_name in dataset_names]) + '    \\\\'
dataset_underline = '    '.join(['\cmidrule(l{\\tabcolsep}r{\\tabcolsep})' + f'{{{2 + (num_shot_settings * num_metrics) * i_col}-{2 + (num_shot_settings * num_metrics) * (i_col + 1) - 1}}}'
                                 for i_col in range(num_datasets)])

shot_row = '{}    &    ' + '  &  '.join([f'\multicolumn{{{num_metrics}}}{{c}}{{{"$n$=" if i == 0 else ""}{task_name_to_shot(task_name)}}}'
                                         for i, task_name in enumerate(pretty_dfs.keys())]) + '    \\\\'
shot_underline = '    '.join(['\cmidrule(l{\\tabcolsep}r{\\tabcolsep})' + f'{{{2 + num_metrics * i_col}-{2 + num_metrics * (i_col + 1) - 1}}}'
                              for i_col in range(num_task)])

print(dataset_row.replace('_', '\_'))
print(dataset_underline.replace('_', '\_'))
print(shot_row.replace('_', '\_'))
print(shot_underline.replace('_', '\_'))


{}    &    \multicolumn{3}{c}{D1-}  &  \multicolumn{3}{c}{D1}  &  \multicolumn{3}{c}{D3}    \\
\cmidrule(l{\tabcolsep}r{\tabcolsep}){2-4}    \cmidrule(l{\tabcolsep}r{\tabcolsep}){5-7}    \cmidrule(l{\tabcolsep}r{\tabcolsep}){8-10}
{}    &    \multicolumn{1}{c}{$n$=10}  &  \multicolumn{1}{c}{100}  &  \multicolumn{1}{c}{1000}  &  \multicolumn{1}{c}{10}  &  \multicolumn{1}{c}{100}  &  \multicolumn{1}{c}{1000}  &  \multicolumn{1}{c}{10}  &  \multicolumn{1}{c}{100}  &  \multicolumn{1}{c}{1000}    \\
\cmidrule(l{\tabcolsep}r{\tabcolsep}){2-2}    \cmidrule(l{\tabcolsep}r{\tabcolsep}){3-3}    \cmidrule(l{\tabcolsep}r{\tabcolsep}){4-4}    \cmidrule(l{\tabcolsep}r{\tabcolsep}){5-5}    \cmidrule(l{\tabcolsep}r{\tabcolsep}){6-6}    \cmidrule(l{\tabcolsep}r{\tabcolsep}){7-7}    \cmidrule(l{\tabcolsep}r{\tabcolsep}){8-8}    \cmidrule(l{\tabcolsep}r{\tabcolsep}){9-9}    \cmidrule(l{\tabcolsep}r{\tabcolsep}){10-10}
